In [1]:
!pip install face_recognition

In [3]:
!pip install opencv-python

In [1]:
import face_recognition #Recognize faces and compare it with the faces already in the database
import cv2 #get input from webcam(opencv) and process it and give it to the face_recognition
import numpy as np
import csv #Handle the csv file 
import os # package to access the files(if have 100 of photos you can automate by using os)
import glob
from datetime import datetime #Get exact date and time 

In [2]:
video_capture = cv2.VideoCapture(0)

In [3]:
# Load images and get face encodings
Bill_image = face_recognition.load_image_file("photos/Bill.jpg")
Bill_encoding = face_recognition.face_encodings(Bill_image)[0]

Elon_image = face_recognition.load_image_file("photos/Elon.jpg")
Elon_encoding = face_recognition.face_encodings(Elon_image)[0]

Mark_image = face_recognition.load_image_file("photos/Mark.jpg")
Mark_encoding = face_recognition.face_encodings(Mark_image)[0]

Steve_image = face_recognition.load_image_file("photos/Steve.jpg")
Steve_encoding = face_recognition.face_encodings(Steve_image)[0]

In [4]:
known_face_encoding = [
    Bill_encoding,  # Corresponds to "Bill"
    Elon_encoding,  # Corresponds to "Elon"
    Mark_encoding,  # Corresponds to "Mark"
    Steve_encoding  # Corresponds to "Steve"
]

known_faces_names = [
    "Bill",  # Matches the first encoding
    "Elon",  # Matches the second encoding
    "Mark",  # Matches the third encoding
    "Steve"  # Matches the fourth encoding
]

In [5]:
students = known_faces_names.copy()

# Initialize variables
face_locations = []# to save face locations
face_encodings = []
face_names = []
s = True

# Get the current date and time
now = datetime.now()
# Format the current date to "YYYY-MM-DD"
current_date = now.strftime("%Y-%m-%d")

f = open(current_date + '.csv', 'w+', newline='')# Open or create a file with the current date as the filename
lnwriter = csv.writer(f) # Initialize CSV writer

In [6]:
while True:
    ret, frame = video_capture.read()  # ERROR 1: Fixed syntax error
    
    # Check if frame was captured successfully
    if not ret:  # ERROR 2: Added frame validation
        print("Failed to capture frame from camera")
        break
    
    # Resize frame for faster processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    # Convert BGR to RGB and ensure proper data type
    rgb_small_frame = np.ascontiguousarray(small_frame[:, :, ::-1].astype(np.uint8))
    
    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
    
    face_names = []
    for face_encoding in face_encodings:
        matches = face_recognition.compare_faces(known_face_encoding, face_encoding)
        face_distance = face_recognition.face_distance(known_face_encoding, face_encoding)
        best_match_index = np.argmin(face_distance)
        
        if matches[best_match_index]:
            name = known_faces_names[best_match_index]
        else:
            name = "Unknown"  # ERROR 3: Handle case when no match is found
            
        face_names.append(name)
        
        # ERROR 4: Moved attendance logic inside the face detection loop and added proper conditions
        if name in students and name != "Unknown":  # Check if the student is in the students list
            students.remove(name)  # Remove the student after marking them present
            print(f"Attendance marked for: {name}")
            print(f"Remaining students: {students}")
            # Record attendance
            current_time = datetime.now().strftime("%H:%M:%S")
            lnwriter.writerow([name, current_time])
            f.flush()  # ERROR 5: Added flush to ensure data is written immediately
            
            # Check if all students are marked present
            if len(students) == 0:
                print("All students marked present! Closing camera...")
                break
    
    # ERROR 6: Added face rectangle drawing for better visualization
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4
        
        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        
        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
    
    # Display video feed
    cv2.imshow("attendance system", frame)
    
    # Break loop on key press
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        print("Exiting...")
        break
    elif key == ord('c'):
        print("Closing after attendance marking...")
        break

# Release video capture and close all windows
video_capture.release()
cv2.destroyAllWindows()
f.close()

Attendance marked for: Elon
Remaining students: ['Bill', 'Mark', 'Steve']
Attendance marked for: Mark
Remaining students: ['Bill', 'Steve']
Attendance marked for: Bill
Remaining students: ['Steve']
Attendance marked for: Steve
Remaining students: []
All students marked present! Closing camera...
Closing after attendance marking...
